In [ ]:
import itkpocus.interson
import itk
import skvideo.io
from pathlib import Path
from ipywidgets import interact, fixed
import matplotlib.pyplot as plt

In [ ]:
# The root data directory
DATA_DIR = Path("../../data/unskilled_operator-20220421_sonivate_interson_fix")

RAW_DIR = DATA_DIR / "raw"
RAW_SONIVATE_DIR = RAW_DIR / "sonivate"
RAW_INTERSON_DIR = RAW_DIR / "interson-spl01"

OUTPUT_SONIVATE_DIR = RAW_SONIVATE_DIR
OUTPUT_INTERSON_DIR = RAW_INTERSON_DIR

In [ ]:
RAW_DIR.exists()

In [ ]:
sonivate_paths = list(RAW_SONIVATE_DIR.glob("**/*.mha"))
interson_paths = list(RAW_INTERSON_DIR.glob("**/*.cine"))
all_paths = sonivate_paths + interson_paths
print(all_paths[:5])

In [ ]:
# Assumes all interson are .cine and all sonivate are .mha
for fp in all_paths:
    print("starting", str(fp))
    
    is_sonivate = fp.suffix == ".mha"

    parent_dir_name = fp.parents[0].name # a digit 1-9
    result_file_name = fp.stem + ".mp4"
    if is_sonivate:
        output_dir = OUTPUT_SONIVATE_DIR
    else:
        output_dir = OUTPUT_INTERSON_DIR

    output_dir /= parent_dir_name
    output_dir.mkdir(exist_ok=True, parents=True)

    if is_sonivate:
        im = itk.imread(str(fp))
    else:
        _, download_settings, _, _, cine_buffer, _, _ = itkpocus.interson._load_video(fp)
        im = itkpocus.interson._convert_to_image(cine_buffer, download_settings.number_of_cine_frames, download_settings.probe_settings.depth, download_settings.frame_scan_rate)

    arr = itk.array_from_image(im)
    skvideo.io.vwrite(str(output_dir / result_file_name), arr)

In [ ]:
# Note we go up to 1000 frames to give enough room. Sliding too far
# gives out of bounds error
def display_arrs_side_by_side(arr1=None, arr2=None, i=(0, 1000, 1)):
    _, axs = plt.subplots(1, 2)
    axs[0].imshow(arr1[i, :, :])
    axs[1].imshow(arr2[i, :, :])

In [ ]:
# For checking the sonivate
sp = sonivate_paths[12]
itk_im = itk.imread(str(sp))
orig_arr = itk.array_from_image(itk_im)

print(sp)

part_num = sp.parents[0].name # a digit 1-9
result_file_name = sp.stem + ".mp4"
path_to_mp4 = OUTPUT_SONIVATE_DIR / part_num / result_file_name
converted_arr = skvideo.io.vread(path_to_mp4)

print(orig_arr.shape, converted_arr.shape)

interact(display_arrs_side_by_side, arr1=fixed(orig_arr), arr2=fixed(converted_arr));

In [ ]:
# For checking the interson
ip = interson_paths[13]
_, download_settings, _, _, cine_buffer, _, _ = itkpocus.interson._load_video(ip)
im = itkpocus.interson._convert_to_image(cine_buffer, download_settings.number_of_cine_frames, download_settings.probe_settings.depth, download_settings.frame_scan_rate)
orig_arr = itk.array_from_image(im)
print(ip)

part_num = ip.parents[0].name # a digit 1-9
result_file_name = ip.stem + ".mp4"
path_to_mp4 = OUTPUT_INTERSON_DIR / part_num / result_file_name
converted_arr = skvideo.io.vread(path_to_mp4)

print(orig_arr.shape, converted_arr.shape)

interact(display_arrs_side_by_side, arr1=fixed(orig_arr), arr2=fixed(converted_arr));